In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [24]:
token_to_id = {
    'what':0,
    'is':1,
    'StatQuest':2,
    'Awesome':3,
    '<EOS>':4,
}

id_to_token = dict(map(reversed, token_to_id.items()))

inputs = torch.tensor([
    [
        token_to_id['what'],
        token_to_id['is'],
        token_to_id['StatQuest'],
        token_to_id['<EOS>'],
    ],
    [
        token_to_id['StatQuest'],
        token_to_id['is'],
        token_to_id['what'],
        token_to_id['<EOS>'],
    ]
])

outputs = torch.tensor([
    [
        token_to_id['is'],
        token_to_id['StatQuest'],
        token_to_id['<EOS>'],
        token_to_id['Awesome'],
        token_to_id['<EOS>'],
    ],
    [
        token_to_id['is'],
        token_to_id['what'],
        token_to_id['<EOS>'],
        token_to_id['Awesome'],
        token_to_id['<EOS>'],
    ]
])

dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset)

In [25]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len = 6, d_model = 2):
        super().__init__()

        pe = torch.zeros(max_len, d_model)

        position = torch.arange(0, max_len, 1).float().unsqueeze(1)
        embedding = torch.arange(0, d_model, 2).float()

        div_term = 1/torch.tensor(1000) ** (embedding/d_model)

        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)

        self.register_buffer('pe', pe)

    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0), :]

Attention (Q, K, V) = SoftMax ((Q.K^T)/sqrt(d_k) + M) V


In [26]:
class Attention(nn.Module):
    def __init__(self, d_model = 2):
        super().__init__()

        self.w_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.w_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.w_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0
        self.col_dim = 1

    def forward(self, embeddings_for_q, embeddings_for_k, embeddings_for_v, mask=None):
        k = self.w_k(embeddings_for_k)
        v = self.w_v(embeddings_for_v)
        q = self.w_q(embeddings_for_q)

        sims = torch.matmul(q, k.tranpose(dim0=self.row_dim, dim1=self.col_dim))
        scaled_sims = sims/torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)

        attnetion_percent = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attnetion_percent, v)
        return attention_scores

In [27]:
class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, max_len=6, d_model=2):
        super().__init__()
        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)
        self.pe = PositionalEncoding(d_model=d_model, max_len=max_len)
        self.self_attention = Attention(d_model=d_model)
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)
        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
        mask = mask == 0
        self_attention_values = self.self_attention(position_encoded, position_encoded, position_encoded, mask=mask)
        residual_connections = position_encoded + self_attention_values
        fc_layer_output = self.fc_layer(residual_connections)
        return fc_layer_output
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = self.loss(output_i, label_i[0])
        return loss

In [28]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)
model_input = torch.tensor([
                        token_to_id['what'],
                        token_to_id['is'],
                        token_to_id['StatQuest'],
                        token_to_id['<EOS>']
                    ])
input_length = model_input.size(dim=0)
predictions = model(input_length)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 6

for i in range(input_length, max_length):
    if(predicted_id == token_to_id['<EOS>']):
        break
    model_input = torch.cat((model_input, predicted_id))
    predictions = model(input_length)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id ))


print("Predicted Tokens:\n")
for id in predicted_ids:
    print('\t', id_to_token[id.item()])


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not int